## References
### Wikipedia
[Iterated Function Systems](https://en.wikipedia.org/wiki/Iterated_function_system)

[Chaos Game](https://en.wikipedia.org/wiki/Chaos_game)

[L-systen](https://en.wikipedia.org/wiki/L-system)

[Barnsley Fern](https://en.wikipedia.org/wiki/Barnsley_fern)

### Python Packages

[turtle graphics](https://docs.python.org/3/library/turtle.html)<br>
Python 3.10 lib/turtle.py documentation

[The Chaos Game](https://beltoforion.de/en/recreational_mathematics/chaos_game.php)<br>
An implementation of the Chaos Game using polygons

## Mathematica
[SubstitutionSystem](https://reference.wolfram.com/language/ref/SubstitutionSystem.html)

In [13]:
from music21 import stream, interval, corpus, instrument, clef, key
from music21 import converter, note, chord, environment, duration, pitch
from music21.meter import TimeSignature
import pandas as pd
import re
from dwbzen.music import MusicUtils, Instruments, MusicScale, MusicSubstitutionSystem, ScoreGen
from dwbzen.common import RuleSet, SubstitutionSystem


In [4]:
mscale = MusicScale(scale_name="Harmonic Minor",root_note=note.Note('C5'))
print(mscale.scale_notes)
print(mscale.formula)
n = mscale.scale_notes[2]
print(n.nameWithOctave)
rn = mscale.get_range_notes(start_octave=4, end_octave=6)
print([x.nameWithOctave for x in rn])

rn = mscale.get_range_notes()
e4 = rn[30]
anote = note.Note("E-4", quarterLength=2)
mscale.get_note(3, anote).nameWithOctave

[<music21.note.Note C>, <music21.note.Note D>, <music21.note.Note E->, <music21.note.Note F>, <music21.note.Note G>, <music21.note.Note G#>, <music21.note.Note B>, <music21.note.Note C>]
[2, 1, 2, 2, 1, 3, 1]
E-5
['C4', 'D4', 'E-4', 'F4', 'G4', 'G#4', 'B4', 'C5', 'D5', 'E-5', 'F5', 'G5', 'G#5', 'B5', 'C6']


'G#4'

In [3]:
mscale.get_note(-10).nameWithOctave

'G3'

## Combined Rules

The previous code has two  rule sets for pitch and duration that were applied independently: the pitch rules first, then the duration rules on the result.  This isn't quite what I had envisioned.

This version combines pitch and duration substitution into a single rule. The downside is the rules are more complex and can no longer be represented by a 2 characters (pitch) or number (duration).<br>
However it is more intuitive and easier to construct logically. For example a rule might be<br>
**replace a given note with 4 notes with pitches 0, +1, -2, +1, and duration half the existing duration**

The pitch rule is expressed as an interval from the previous pitch as before.<br>
The duration rule is expressed as a multiple of the duration of the previous note.

In addition each substitution has optional pre- and post- processing rule(s). The duration post-processing rule for example
would check if the resulting duration is less than, say, a 16th note (0.25 quarterLength) and if so set to 1 quarterLength.<br>
The RuleSet class then will consist of the following:
1. Pre-processing rules expressed as functions.
2. Pitch, duration substitutions as an ordered pair: (pitch, duration). For example ('0+1-2+1', 0.5)
3. Post-processing rules expressed as function(s).

Rule application is recursive: each new generation is created by applying the rule set to the previous generation.

In [7]:
#
# test RuleSet
#
substitution_rules = {\
        r'(?P<interval>[+-]?0)/(?P<duration>\d+\.\d+)':['0/0.5', '+1/1.0', '-2/1.0', '+1/2.0'], \
        r'(?P<interval>[+-]?1)/(?P<duration>\d+\.\d+)' : ['0/<duration>', '1/0.5']\
}
ruleSet = RuleSet(substitution_rules)
print(f'substitutions: {ruleSet.substitutions}\n')
for subs in ruleSet.substitutions:
    print(f'sub: {subs}')
    print(f' pattern: {subs["pattern"]}\n replacement: {subs["replacement"]}\n' )

substitutions: [{'pattern': re.compile('(?P<interval>[+-]?0)/(?P<duration>\\d+\\.\\d+)'), 'replacement': ['0/0.5', '+1/1.0', '-2/1.0', '+1/2.0']}, {'pattern': re.compile('(?P<interval>[+-]?1)/(?P<duration>\\d+\\.\\d+)'), 'replacement': ['0/<duration>', '1/0.5']}]

sub: {'pattern': re.compile('(?P<interval>[+-]?0)/(?P<duration>\\d+\\.\\d+)'), 'replacement': ['0/0.5', '+1/1.0', '-2/1.0', '+1/2.0']}
 pattern: re.compile('(?P<interval>[+-]?0)/(?P<duration>\\d+\\.\\d+)')
 replacement: ['0/0.5', '+1/1.0', '-2/1.0', '+1/2.0']

sub: {'pattern': re.compile('(?P<interval>[+-]?1)/(?P<duration>\\d+\\.\\d+)'), 'replacement': ['0/<duration>', '1/0.5']}
 pattern: re.compile('(?P<interval>[+-]?1)/(?P<duration>\\d+\\.\\d+)')
 replacement: ['0/<duration>', '1/0.5']



In [10]:
start = ['0/1.0', '+1/0.5']
substitution_rules = {\
        r'(?P<interval>[+-]?[0123])/(?P<duration>\d+\.\d+)':['0/0.5', '+3/1.0', '-2/1.0', '-1/0.5'] } #, \
        #r'(?P<interval>[+-]?1)/(?P<duration>\d+\.\d+)' : ['0/1.0', '1/0.5']   }
ruleSet = RuleSet(substitution_rules)
ss = MusicSubstitutionSystem(ruleSet, verbose=0)

In [11]:
sub_result = ss.apply(start,2)
print(f'{len(sub_result)} commands:\n{sub_result} ')

32 commands:
['0/0.5', '+3/1.0', '-2/1.0', '-1/0.5', '0/0.5', '+3/1.0', '-2/1.0', '-1/0.5', '0/0.5', '+3/1.0', '-2/1.0', '-1/0.5', '0/0.5', '+3/1.0', '-2/1.0', '-1/0.5', '0/0.5', '+3/1.0', '-2/1.0', '-1/0.5', '0/0.5', '+3/1.0', '-2/1.0', '-1/0.5', '0/0.5', '+3/1.0', '-2/1.0', '-1/0.5', '0/0.5', '+3/1.0', '-2/1.0', '-1/0.5'] 


In [12]:
import re
# the generic match string - splitter
re_str = r'(?P<interval>[+-]?\d+)/(?P<duration>\d+\.\d+)'
re = re.compile(re_str)
m = re.match('0/0.5')
print(m.groups())
m = re.match('+1/1.0')
print(m.groupdict()['duration'])

('0', '0.5')
1.0


## Generate a Part 
ScoreGen class creates a music21.Part from the commands output of MusicSubstitutionSystem

### Rules code

In [9]:
from music21 import note, duration
from music21.stream import Score

trace = False

def show(anote:note.Note):
    print(f"pitch: {anote.nameWithOctave}  duration: {anote.duration.quarterLength}")
    
def score_postprocess(ascore:Score, instrument_name:str):
    '''Checks the pitch against the range of the given instrument, adjusting if necessary.
    '''
    if trace:
        print('score_postprocess')


def score_preprocess(ascore:Score):
    if trace:
        print('score_preprocess')

def interval_rule(sd:str, state):
    anote = state['note']

    if sd == 'interval':
        scale_degrees = 0
    else:
        scale_degrees = int(sd)
        
    if trace:
        print(f'apply interval {scale_degrees} TO  {anote.nameWithOctave} {anote.duration}')
        
    musicScale = state['musicScale']
    note_dur = anote.duration.quarterLength
    scale_note = musicScale.get_note(scale_degrees, anote)
    next_note = copy.deepcopy(scale_note)
    next_note.duration = duration.Duration(note_dur)
    state['previous_note'] = anote
    state['note'] = next_note
    #
    return next_note

def duration_rule(d:str, state):
    '''Apply the duration command d to Note anote.
        Arguments:
            d - a float to multiply the Notes duration.quarterLength by
            anote - a Note
        Note if the resulting product is >4, it is reduced by half.
        If < 0.125, it is multiplied by 2
    '''
    anote = state['note']
    if d=='duration':
        multiplier = 1.0
    else:
        multiplier = float(d)
        
    if trace:
        print(f'apply duration {multiplier} TO {anote.nameWithOctave} {anote.duration}')
        
    old_dur = anote.duration.quarterLength
    new_dur = round(old_dur * multiplier, 3)
    if new_dur > 4.0:      # the maximum duration is a whole note
        new_dur = 2.0    
    elif new_dur < 0.125:  # the minimum duration is 32nd note
        new_dur = 0.25
        
    new_note = note.Note(anote.nameWithOctave, quarterLength=new_dur)
    if trace:
        print(f"\told_dur: {old_dur}  new_dur: {new_dur}\n")
    state['note'] = new_note
    return new_note


In [16]:
import re

substitution_rules = {\
        r'(?P<interval>[+-]?0)/(?P<duration>\d+\.\d+)':['0/0.5', '+2/0.5', '-3/1.0', '+1/2.0'], \
        r'(?P<interval>[+-]?1)/(?P<duration>\d+\.\d+)' : ['2/duration', '-1/2.0'] #, \
        #r'(?P<interval>[+-]?2)/(?P<duration>\d+\.\d+)' : ['+2/1.0', '+2/2.0, +1/0.5']
}
command_rules = {'interval':interval_rule, 'duration':duration_rule}
rules = {'commands':command_rules}
splitter = r'(?P<interval>[+-]?\d+)/(?P<duration>\d+\.\d+)'
rule_set = RuleSet(substitution_rules, rules=rules, splitter=splitter)
start = ['0/1.0', '+2/0.5', '-3/0.5', '1/2.0']

ss = MusicSubstitutionSystem(rule_set, verbose=0)
commands = ss.apply(start,1)
print(f'{len(commands)} commands:\n{commands} ')

12 commands:
['0/0.5', '+2/0.5', '-3/1.0', '+1/2.0', '+2/0.5', '-3/0.5', '1/2.0', '0/1.0', '+2/0.5', '-3/0.5', '2/duration', '-1/2.0'] 


In [17]:
score_gen = ScoreGen(rule_set, scale_name='Harmonic Minor', instrument_name='Soprano', key=key.Key('c'), verbose=0)
start_note=note.Note("C5", quarterLength=2)
ascore = score_gen.run(commands, start_note=start_note)


In [18]:
ascore.show('musicxml')

## Test code

In [29]:
# splitter match string
import re

rule_re_str = r'(?P<interval>[+-]?\d+)/(?P<duration>\d+\.\d+)'
rule_re = re.compile(rule_re_str)
m = rule_re.match('+1/1.0')
gd = m.groupdict()
print(gd)
print(int(gd['interval']))
print(float(gd['duration']))

{'interval': '+1', 'duration': '1.0'}
1
1.0


In [18]:
# match string for specific rule
import re

rule1_re_str = r'(?P<interval>[+-]?0)/(?P<duration>\d+\.\d+)'
rule1_re = re.compile(rule1_re_str)   # re.Pattern
rule2_re_str = r'(?P<interval>[+-]?1)/(?P<duration>\d+\.\d+)'
rule2_re = re.compile(rule2_re_str)
m1 = rule1_re.match('0/0.5')
print(m1.groupdict())

m2 = rule2_re.match('+1/1.0')
if m2 is not None:
    print(m2.groupdict())

{'interval': '0', 'duration': '0.5'}
{'interval': '+1', 'duration': '1.0'}


## Pitch/Duration-based rule set

Rules use pitches (name + octave) and quarterLength durations. Examples:<br>
**'C5/2.0' -->  'E5/1.0', 'B4/0.5', 'D5/0.5', 'C5/1.0'**

Regular expressions allows for matching the pitch only (i.e. no octave).<br>
Grouping Tags are \<pitch\>,  '\<octave\>' and '\<quarterLength\>'
    

In [20]:
import re
#
# Rules may have specific pitches with or without an octave.
#
ms = r'(?P<pitch>[A-Ga-g])(?P<octave>\d?)/(?P<quarterLength>\d+\.\d+)'  # the RuleSet splitter
ms_re = re.compile(ms1)
ms_match = ms_re.match('C4/0.5')
print(ms_match.groupdict())

ms_match = ms_re.match('C/1.0')
print(ms_match.groupdict())  # match any C pitch

{'pitch': 'C', 'octave': '4', 'quarterLength': '0.5'}
{'pitch': 'C', 'octave': '', 'quarterLength': '1.0'}


In [21]:
#
# substitution rules can use discrete pitches or reference group tags
# The ScoreGen rules substitute tags for real values based on the input
#
substitution_rules = {\
    r'(?P<pitch>[A-Ga-g])(?P<octave>\d?)/(?P<quarterLength>\d+\.\d+)' : \
        [ 'pitch+3/quarterLength*0.5', 'pitch-1/quarterLength*0.25', 'pitch+1/quarterLength*0.25', 'pitch/quarterLength*0.5' ],
    r'(?P<pitch>C)(?P<octave>5)/(?P<quarterLength>\d+\.\d+)' : \
        [ 'E5/quarterLength*0.5', 'B4/quarterLength*0.25', 'D5/quarterLength*0.25', 'C5/quarterLength*0.5' ]
}

### Rules Code

In [ ]:
def pitch_rule(sd:str, state):
    anote = state['note']
    

def quarterLength_rule(ql:str, state):
    pass



In [2]:
substitution_rules = {\
    r'(?P<interval>[+-]?[01])/(?P<duration>\d+\.\d+)':['0/0.5', '+3/1.0', '-2/2.0', '-1/2.0']  , \
    r'(?P<interval>[+-]?[23])/(?P<duration>\d+\.\d+)' : ['-1/1.0', '-2/2.0', '+2/0.5', '0/1.0']
}

In [3]:
substitution_rules

{'(?P<interval>[+-]?[01])/(?P<duration>\\d+\\.\\d+)': ['0/0.5',
  '+3/1.0',
  '-2/2.0',
  '-1/2.0'],
 '(?P<interval>[+-]?[23])/(?P<duration>\\d+\\.\\d+)': ['-1/1.0',
  '-2/2.0',
  '+2/0.5',
  '0/1.0']}

In [13]:
import json, math
filename = "resources/music/substitution_rules.json"
fp = open(filename, "r")
jtxt = fp.read()
jdoc = json.loads(jtxt)

In [5]:
jdoc['rule1']

{'(?P<interval>[+-]?[01])/(?P<duration>\\d+\\.\\d+)': ['0/0.5',
  '+3/1.0',
  '-2/2.0',
  '-1/2.0'],
 '(?P<interval>[+-]?[23])/(?P<duration>\\d+\\.\\d+)': ['-1/1.0',
  '-2/2.0',
  '+2/0.5',
  '0/1.0']}

In [18]:
maxlen=26.0
plen = 18.5
math.ceil(maxlen/4) * 4 - plen

9.5

In [23]:
ts = TimeSignature('4/4')

In [24]:
ts.numerator * ts.beatDuration.quarterLength

4.0